# Capstone Project - The Battle of Neighborhoods

### Introduction

My friend has plan to move to either Manhattan,NYC or Toronto. He want to live there and do some business but he has no idea, which business or which location. So, in this project we gonna help him analyze these 2 cities and give him some idea.  

We gonna compare city of Manhattan and Toronto .They are very diverse and are the financial capitals of their respective countries. We would like compare the neighborhoods of the two cities and determine how similar or dissimilar they are

### Data

For Toronto data we are going to get from wikipedia : "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 

And lat,long data from http://cocl.us/Geospatial_data

For NewYork , will get from : https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json

After that we will get the venue information of each area from FourSquare API. Then we can do segmentation and clustering them.

#### let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import requests

from bs4 import BeautifulSoup

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  19.01 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  19.18 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.44 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  16.34 MB/s
vincent-0.4.4- 100% |###################

### Load and Explore the data

Download the data of New York into Dataframe

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_df = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_df = ny_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
ny_df.head()    

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let's focus only Manhattan

In [3]:
manhattan_df = ny_df[ny_df['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### Let's download data for Toronto

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_page = requests.get(url).text
soup = BeautifulSoup(wiki_page,'lxml')

list1 = []
list2 = []
list3 = []


table_body = soup.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]

    if cols and (cols[1] != 'Not assigned')  :
        
        if (cols[0]!=prev_col):       
            list1.append(cols[0])
            list2.append(cols[1])
        
            if (cols[2] == 'Not assigned'):
                cols[2] = cols[1]
            list3.append(cols[2])
            
        else:
            last_index = len(list3)-1
            list3[last_index] = list3[last_index] + ", " + cols[2]
            
    if cols:
        prev_col = cols[0]
        

postcode_df = pd.DataFrame({'Postcode': list1, 'Borough': list2, 'Neighborhood': list3})
postcode_df = postcode_df[['Postcode','Borough','Neighborhood']]

del list1, list2, list3

postcode_df.head() 

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Next, import csv file for latitude and the longitude coordinates and merge two dataframes.

In [5]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data

geo_data = pd.read_csv("Geospatial_Coordinates.csv", delimiter=",")
geo_data.columns = ['Postcode', 'Latitude','Longitude']
#geo_data.head()


canada_df = pd.merge(postcode_df, geo_data, how='left', on=['Postcode'])
canada_df.head()

--2019-01-04 08:40:28--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-01-04 08:40:28--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-01-04 08:40:30--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-01-04 08:40:30--  https://ibm.ent.box.com/shared/

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Filter only Toronto from dataframe

In [6]:
toronto_df = canada_df[canada_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_df = toronto_df[['Borough','Neighborhood','Latitude','Longitude']]
toronto_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,East Toronto,The Beaches,43.676357,-79.293031
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


# Plotting Maps of Toronto and Manhattan

Use geopy library to get the latitude and longitude of Toronto.

In [7]:
address = 'Toronto , TO'

geolocator = Nominatim()
location = geolocator.geocode(address)
t_latitude = location.latitude
t_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(t_latitude, t_longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


let's visualizat Toronto the neighborhoods in it.

In [8]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next , do the same with Manhattan

In [9]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
m_latitude = location.latitude
m_longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(m_latitude, m_longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [10]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[m_latitude, m_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_df['Latitude'], manhattan_df['Longitude'], manhattan_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Methodology

In this project, We will use the basic methodology.</br>

We have plotted maps of Manhattan and Toronto using latitude and longitude values.

After this we will use the Foursquare API to explore neighborhoods in both cities.

We will use explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters

K-means clustering algorithm will be used to complete this task.

#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = '0RXOHFFZU5TGZMGB0ZA5VKZ21NKLAQUOGTOLIWBVSYI1EHBE' # your Foursquare ID
CLIENT_SECRET = 'DY2WSEMAJA1GIAY1GFYT1QYN4SR4VFDOF0BPUZHORLYK5T5T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0RXOHFFZU5TGZMGB0ZA5VKZ21NKLAQUOGTOLIWBVSYI1EHBE
CLIENT_SECRET:DY2WSEMAJA1GIAY1GFYT1QYN4SR4VFDOF0BPUZHORLYK5T5T


Let's create a function to explore the neighborhoods

In [12]:
#default radius is 500 meters and limit is 10
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [13]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

Let's check the size of the resulting dataframe

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(338, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood

In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"Brockton, Exhibition Place, Parkdale Village",10,10,10,10,10,10
Business reply mail Processing Centre969 Eastern,10,10,10,10,10,10
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",10,10,10,10,10,10
"Cabbagetown, St. James Town",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
"Chinatown, Grange Park, Kensington Market",10,10,10,10,10,10
Christie,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 117 uniques categories.


### Analyze Each Neighborhood

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tea Room,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tea Room,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.1,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.1,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.100000,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
3,Business reply mail Processing Centre969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.000000,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.00,0.0,0.1,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.1,0.1,0.1,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [20]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Noodle House,Coffee Shop,Plaza,Concert Hall,Hotel,Speakeasy,Greek Restaurant,Café,Dog Run
1,Berczy Park,Cocktail Bar,Farmers Market,Museum,Concert Hall,Italian Restaurant,Steakhouse,Beer Bar,Liquor Store,Park,Thai Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Furniture / Home Store,Bar,Gym,Italian Restaurant,Café,Pet Store,Breakfast Spot,Caribbean Restaurant,Arts & Crafts Store
3,Business reply mail Processing Centre969 Eastern,Burrito Place,Pizza Place,Garden Center,Brewery,Skate Park,Fast Food Restaurant,Farmers Market,Restaurant,Comic Shop,Auto Workshop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Plane,Cosmetics Shop,Dance Studio
5,"Cabbagetown, St. James Town",Indian Restaurant,Café,Butcher,General Entertainment,Diner,Japanese Restaurant,Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar
6,Central Bay Street,Coffee Shop,Ramen Restaurant,Italian Restaurant,Sushi Restaurant,Gastropub,Spa,Pizza Place,Vegetarian / Vegan Restaurant,Comic Shop,Concert Hall
7,"Chinatown, Grange Park, Kensington Market",Café,Organic Grocery,Mexican Restaurant,Dessert Shop,Arts & Crafts Store,Bar,Coffee Shop,Cocktail Bar,College Gym,Cuban Restaurant
8,Christie,Café,Grocery Store,Diner,Restaurant,Baby Store,Italian Restaurant,Coffee Shop,College Gym,Comfort Food Restaurant,Creperie
9,Church and Wellesley,Salon / Barbershop,Park,Ramen Restaurant,Bookstore,Dance Studio,Breakfast Spot,Bubble Tea Shop,Bar,Gastropub,Tea Room


#### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 4 clusters.

In [21]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 1, 3, 1, 3, 1, 1, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [22]:
toronto_merged = toronto_df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,Breakfast Spot,Historic Site,Bakery,Coffee Shop,Park,Pub,Gym / Fitness Center,Spa,Restaurant,Convenience Store
1,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,3,Burrito Place,Ramen Restaurant,Clothing Store,Pizza Place,Comic Shop,Plaza,Café,Thai Restaurant,Tea Room,Taco Place
2,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Food Truck,Japanese Restaurant,Church,Restaurant,Italian Restaurant,BBQ Joint,Creperie,Coffee Shop,Gastropub,Middle Eastern Restaurant
3,East Toronto,The Beaches,43.676357,-79.293031,1,Music Venue,Coffee Shop,Pub,Vegetarian / Vegan Restaurant,Cheese Shop,Church,Clothing Store,Cocktail Bar,College Gym,Comfort Food Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Cocktail Bar,Farmers Market,Museum,Concert Hall,Italian Restaurant,Steakhouse,Beer Bar,Liquor Store,Park,Thai Restaurant


Finally, let's visualize the resulting clusters

In [23]:
# create map
toronto_map_clusters = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters

### Analyze Manhattan

Now write the code to explore neighborhood and create a new dataframe called manhattan_venues.

In [24]:
manhattan_venues = getNearbyVenues(names=manhattan_df['Neighborhood'],
                                   latitudes=manhattan_df['Latitude'],
                                   longitudes=manhattan_df['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Let's check the size of the resulting dataframe

In [25]:
print(manhattan_venues.shape)
manhattan_venues.head()

(400, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


Let's check how many venues were returned for each neighborhood

In [26]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,10,10,10,10,10,10
Carnegie Hill,10,10,10,10,10,10
Central Harlem,10,10,10,10,10,10
Chelsea,10,10,10,10,10,10
Chinatown,10,10,10,10,10,10
Civic Center,10,10,10,10,10,10
Clinton,10,10,10,10,10,10
East Harlem,10,10,10,10,10,10
East Village,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [66]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 153 uniques categories.


####  Analyze Each Neighborhood in Manhattan

In [27]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Trail,Bookstore,Boutique,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,General Entertainment,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,School,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Trail,Bookstore,Boutique,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,General Entertainment,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,School,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Carnegie Hill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0
2,Central Harlem,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

#### Let's put that into a pandas dataframe and display the top 10 venues for each neighborhood.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Food Court,Gym,Performing Arts Venue,Plaza,Coffee Shop,Shopping Mall,Smoke Shop,Market,Park,BBQ Joint
1,Carnegie Hill,Gym / Fitness Center,Wine Shop,Wine Bar,Dance Studio,Community Center,Pizza Place,Italian Restaurant,Bookstore,Gym,Donut Shop
2,Central Harlem,African Restaurant,American Restaurant,Cycle Studio,History Museum,Library,Bar,French Restaurant,Beer Bar,Ethiopian Restaurant,Music Venue
3,Chelsea,Indian Restaurant,Beer Bar,Café,Cupcake Shop,Vegetarian / Vegan Restaurant,Nightclub,Asian Restaurant,New American Restaurant,Theater,Speakeasy
4,Chinatown,Yoga Studio,Chinese Restaurant,Greek Restaurant,General Entertainment,Museum,English Restaurant,Noodle House,Cocktail Bar,Sandwich Place,Hotel
5,Civic Center,Spa,Gym / Fitness Center,Dance Studio,Falafel Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,French Restaurant,Bar,Bakery,Cuban Restaurant
6,Clinton,Theater,Peruvian Restaurant,Comedy Club,Building,Gym,Gym / Fitness Center,Café,Hotel,English Restaurant,Filipino Restaurant
7,East Harlem,French Restaurant,Thai Restaurant,Sandwich Place,Spanish Restaurant,Steakhouse,Latin American Restaurant,Beer Bar,Mexican Restaurant,Coffee Shop,Pharmacy
8,East Village,Ice Cream Shop,Beer Store,Hot Dog Joint,American Restaurant,Moroccan Restaurant,Dog Run,Bagel Shop,Japanese Restaurant,Coffee Shop,Korean Restaurant
9,Financial District,Coffee Shop,Gym / Fitness Center,Event Space,Steakhouse,New American Restaurant,Gym,Monument / Landmark,Historic Site,Pizza Place,Department Store


#### Cluster Neighborhoods in Manhattan

Run *k*-means to cluster the neighborhood into 4 clusters.

In [30]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 2, 1, 3, 2, 0, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
manhattan_merged = manhattan_df

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Coffee Shop,Yoga Studio,Seafood Restaurant,Donut Shop,Tennis Stadium,Diner,Department Store,Gym,Pizza Place,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,0,Yoga Studio,Chinese Restaurant,Greek Restaurant,General Entertainment,Museum,English Restaurant,Noodle House,Cocktail Bar,Sandwich Place,Hotel
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Burger Joint,Bakery,Ramen Restaurant,Restaurant,Breakfast Spot,Market,Park,Italian Restaurant,Hawaiian Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,2,Park,Deli / Bodega,Farmers Market,Latin American Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Pet Store,Wine Bar,Wine Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Cocktail Bar,Italian Restaurant,Café,Caribbean Restaurant,Bar,Smoke Shop,Yoga Studio,Wine Bar,Mexican Restaurant,Dog Run


Finally, let's visualize the resulting clusters

In [32]:
# create map
manhattan_map_clusters = folium.Map(location=[m_latitude, m_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(manhattan_map_clusters)
       
manhattan_map_clusters

## Results

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Toronto Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Lawrence Park,0,Park,Swim School,Bus Line,Lake,Dim Sum Restaurant,Vegetarian / Vegan Restaurant,Cosmetics Shop,Church,Clothing Store,Cocktail Bar
22,North Toronto West,0,Sporting Goods Shop,Yoga Studio,Mexican Restaurant,Coffee Shop,Salon / Barbershop,Spa,Diner,Chinese Restaurant,Fast Food Restaurant,Eastern European Restaurant
24,"Parkdale, Roncesvalles",0,Cuban Restaurant,Gift Shop,Bar,Italian Restaurant,Dessert Shop,Dog Run,Eastern European Restaurant,Coffee Shop,Movie Theater,Restaurant
27,"Runnymede, Swansea",0,Sushi Restaurant,Burrito Place,Café,Tea Room,Food,Bookstore,Fish & Chips Shop,Pub,Coffee Shop,Comfort Food Restaurant


#### Toronto Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,The Beaches,1,Music Venue,Coffee Shop,Pub,Vegetarian / Vegan Restaurant,Cheese Shop,Church,Clothing Store,Cocktail Bar,College Gym,Comfort Food Restaurant
5,Central Bay Street,1,Coffee Shop,Ramen Restaurant,Italian Restaurant,Sushi Restaurant,Gastropub,Spa,Pizza Place,Vegetarian / Vegan Restaurant,Comic Shop,Concert Hall
7,"Adelaide, King, Richmond",1,Steakhouse,Noodle House,Coffee Shop,Plaza,Concert Hall,Hotel,Speakeasy,Greek Restaurant,Café,Dog Run
8,"Dovercourt Village, Dufferin",1,Bakery,Bank,Gym / Fitness Center,Middle Eastern Restaurant,Café,Brewery,Bar,Music Venue,Supermarket,Farmers Market
10,"Little Portugal, Trinity",1,Bar,Greek Restaurant,Pizza Place,Korean Restaurant,Brewery,Ice Cream Shop,Cuban Restaurant,Asian Restaurant,New American Restaurant,Dog Run
11,"The Danforth West, Riverdale",1,Greek Restaurant,Yoga Studio,Ice Cream Shop,Brewery,Cosmetics Shop,Health Food Store,Italian Restaurant,Cocktail Bar,Coffee Shop,College Gym
14,"The Beaches West, India Bazaar",1,Gym,Italian Restaurant,Fish & Chips Shop,Pub,Liquor Store,Sushi Restaurant,Park,Ice Cream Shop,Burger Joint,Brewery
15,"Commerce Court, Victoria Hotel",1,Café,Gastropub,Museum,Gym,American Restaurant,Art Gallery,Restaurant,Gym / Fitness Center,Cosmetics Shop,Convenience Store
16,Studio District,1,Coffee Shop,Ice Cream Shop,Sandwich Place,Café,Comfort Food Restaurant,Bookstore,Cheese Shop,Fish Market,Farmers Market,Clothing Store
18,Roselawn,1,Garden,Dance Studio,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall


#### Toronto Cluster 3

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St. James Town,2,Food Truck,BBQ Joint,Middle Eastern Restaurant,Coffee Shop,Japanese Restaurant,Italian Restaurant,Restaurant,Creperie,Church,Gastropub
6,Christie,2,Café,Grocery Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Diner,Concert Hall,Cosmetics Shop,Convenience Store
13,"Brockton, Exhibition Place, Parkdale Village",2,Coffee Shop,Caribbean Restaurant,Bar,Gym,Italian Restaurant,Café,Pet Store,Breakfast Spot,Furniture / Home Store,Arts & Crafts Store
17,Lawrence Park,2,Park,Bus Line,Swim School,Photography Studio,Vegetarian / Vegan Restaurant,Cosmetics Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar
20,"Forest Hill North, Forest Hill West",2,Trail,Park,Sushi Restaurant,Jewelry Store,Vegetarian / Vegan Restaurant,Creperie,Church,Clothing Store,Cocktail Bar,Coffee Shop
25,Davisville,2,Dessert Shop,Café,Coffee Shop,Indian Restaurant,Seafood Restaurant,Italian Restaurant,Pizza Place,Sushi Restaurant,Park,Cocktail Bar
29,"Chinatown, Grange Park, Kensington Market",2,Café,Organic Grocery,Bar,Cocktail Bar,Coffee Shop,Mexican Restaurant,Arts & Crafts Store,Dessert Shop,Clothing Store,College Gym
33,Stn A PO Boxes 25 The Esplanade,2,Cocktail Bar,Museum,Thai Restaurant,Food Truck,Café,Beer Bar,Park,Concert Hall,Steakhouse,College Gym
35,"First Canadian Place, Underground city",2,Café,American Restaurant,Restaurant,Gym / Fitness Center,Gym,Art Gallery,Steakhouse,Bakery,Gastropub,Fish & Chips Shop
36,Church and Wellesley,2,Salon / Barbershop,Park,Bookstore,Dance Studio,Breakfast Spot,Bubble Tea Shop,Bar,Ramen Restaurant,Gastropub,Tea Room


#### Toronto Cluster 4

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"Parkdale, Roncesvalles",3,Cuban Restaurant,Eastern European Restaurant,Restaurant,Italian Restaurant,Dessert Shop,Dog Run,Bar,Gift Shop,Coffee Shop,Movie Theater


#### Manhattan Cluster 1

In [96]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,0,Coffee Shop,Yoga Studio,Seafood Restaurant,Donut Shop,Tennis Stadium,Diner,Department Store,Gym,Pizza Place,Deli / Bodega
1,Chinatown,0,Yoga Studio,Greek Restaurant,Hotel,Museum,Chinese Restaurant,Sandwich Place,Noodle House,Cocktail Bar,General Entertainment,English Restaurant
7,East Harlem,0,French Restaurant,Thai Restaurant,Sandwich Place,Spanish Restaurant,Steakhouse,Latin American Restaurant,Beer Bar,Mexican Restaurant,Coffee Shop,Pharmacy
9,Yorkville,0,Wine Shop,Sushi Restaurant,Park,Liquor Store,Deli / Bodega,Sandwich Place,Asian Restaurant,Bagel Shop,Diner,Yoga Studio
10,Lenox Hill,0,Smoke Shop,Women's Store,Art Gallery,Health Food Store,College Academic Building,Pizza Place,Liquor Store,Thai Restaurant,Salad Place,Taco Place
11,Roosevelt Island,0,Outdoors & Recreation,Deli / Bodega,Park,School,Liquor Store,Sandwich Place,Residential Building (Apartment / Condo),Greek Restaurant,Waterfront,Farmers Market
15,Midtown,0,Park,Hotel,French Restaurant,Christmas Market,Salad Place,Chinese Restaurant,Smoke Shop,Spa,Cuban Restaurant,Event Space
19,East Village,0,Ice Cream Shop,Hot Dog Joint,American Restaurant,Moroccan Restaurant,Japanese Restaurant,Bagel Shop,Dog Run,Korean Restaurant,Coffee Shop,Beer Store
20,Lower East Side,0,Coffee Shop,Italian Restaurant,Filipino Restaurant,Chinese Restaurant,Cocktail Bar,Community Center,Yoga Studio,Art Gallery,Vietnamese Restaurant,Diner
22,Little Italy,0,Café,Wine Bar,Spanish Restaurant,Sandwich Place,Coffee Shop,Salad Place,Gourmet Shop,Yoga Studio,Donut Shop,Event Space


#### Manhattan Cluster 2

In [97]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,1,Cocktail Bar,Yoga Studio,Italian Restaurant,Caribbean Restaurant,Café,Smoke Shop,Bar,Mexican Restaurant,Wine Bar,Hot Dog Joint
12,Upper West Side,1,Southern / Soul Food Restaurant,Italian Restaurant,Chinese Restaurant,Bookstore,Movie Theater,Pub,Café,Bagel Shop,Juice Bar,American Restaurant
13,Lincoln Square,1,Indie Movie Theater,Concert Hall,Performing Arts Venue,Indie Theater,School,Opera House,Theater,Library,Dog Run,Event Space
18,Greenwich Village,1,Mexican Restaurant,Sushi Restaurant,Snack Place,Caribbean Restaurant,Optical Shop,French Restaurant,Italian Restaurant,Bagel Shop,Jazz Club,Café
21,Tribeca,1,Park,Sushi Restaurant,Hotel,American Restaurant,Cycle Studio,Indie Theater,Spa,Italian Restaurant,Salad Place,Steakhouse
24,West Village,1,Italian Restaurant,Gastropub,Chinese Restaurant,Accessories Store,American Restaurant,Speakeasy,Coffee Shop,Ethiopian Restaurant,Filipino Restaurant,Farmers Market
27,Gramercy,1,Coffee Shop,Yoga Studio,Playground,Cheese Shop,Beer Bar,Restaurant,Food Truck,Israeli Restaurant,Irish Pub,Department Store
30,Carnegie Hill,1,Gym / Fitness Center,Wine Shop,Wine Bar,Community Center,Pizza Place,Dance Studio,Italian Restaurant,Bookstore,Gym,Dog Run
31,Noho,1,Deli / Bodega,Italian Restaurant,Southern / Soul Food Restaurant,Ice Cream Shop,American Restaurant,Cocktail Bar,Gourmet Shop,Rock Club,English Restaurant,Filipino Restaurant
34,Sutton Place,1,Yoga Studio,Italian Restaurant,Steakhouse,Grocery Store,Beer Store,Beer Garden,Gym,Mexican Restaurant,Adult Boutique,American Restaurant


#### Manhattan Cluster 3

In [98]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,2,Pet Store,Wine Shop,Wine Bar,Farmers Market,Park,Deli / Bodega,Coffee Shop,Mexican Restaurant,Bakery,Latin American Restaurant
6,Central Harlem,2,French Restaurant,Music Venue,African Restaurant,American Restaurant,Cycle Studio,History Museum,Bar,Library,Beer Bar,Ethiopian Restaurant
17,Chelsea,2,Café,Beer Bar,Indian Restaurant,Cupcake Shop,Vegetarian / Vegan Restaurant,Asian Restaurant,Nightclub,Theater,New American Restaurant,Speakeasy


#### Manhattan Cluster 4

In [99]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,3,Café,Burger Joint,Bakery,Ramen Restaurant,Restaurant,Breakfast Spot,Market,Park,Italian Restaurant,Hawaiian Restaurant
5,Manhattanville,3,Park,BBQ Joint,Climbing Gym,Bike Trail,Supermarket,Ramen Restaurant,Italian Restaurant,Juice Bar,Asian Restaurant,Other Nightlife
8,Upper East Side,3,Hotel Bar,Italian Restaurant,Burrito Place,Chocolate Shop,Boutique,Bakery,Café,Vegetarian / Vegan Restaurant,Hotel,American Restaurant
14,Clinton,3,Theater,Peruvian Restaurant,Comedy Club,Building,Gym / Fitness Center,Gym,Café,Hotel,Donut Shop,Farmers Market
16,Murray Hill,3,Cocktail Bar,Hotel,Spa,Vegetarian / Vegan Restaurant,Hawaiian Restaurant,Japanese Restaurant,Jazz Club,Coffee Shop,Tea Room,Ramen Restaurant
23,Soho,3,Women's Store,Cycle Studio,Arts & Crafts Store,Men's Store,Salon / Barbershop,Dessert Shop,Clothing Store,Dance Studio,Miscellaneous Shop,Department Store
25,Manhattan Valley,3,Coffee Shop,Wine Shop,Fried Chicken Joint,Hostel,Vietnamese Restaurant,Deli / Bodega,Arts & Crafts Store,Italian Restaurant,Bar,Korean Restaurant
28,Battery Park City,3,Park,BBQ Joint,Food Court,Market,Shopping Mall,Coffee Shop,Plaza,Gym,Smoke Shop,Performing Arts Venue
29,Financial District,3,Coffee Shop,Monument / Landmark,New American Restaurant,Pizza Place,Event Space,Historic Site,Steakhouse,Gym / Fitness Center,Gym,Falafel Restaurant
32,Civic Center,3,Spa,Gym / Fitness Center,Dance Studio,Falafel Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,French Restaurant,Bar,Bakery,Cuban Restaurant


### Thank you. Discussion and Conclusion will be in the report.